# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse     Shipments  \
0  Sep  6 2022  1:31PM  245759        12      HH-36786   
1  Sep  6 2022  1:31PM  245759        12      HH-36787   
2  Sep  6 2022  1:31PM  245759        12      HH-36788   
3  Sep  6 2022  1:31PM  245759        12      HH-36789   
4  Sep  6 2022  1:31PM  245759        12      HH-36790   
5  Sep  6 2022  1:31PM  245759        12      HH-36791   
6  Sep  6 2022  1:27PM  245758        10       8492666   
7  Sep  6 2022  1:20PM  245753        10  SOTAH0000771   
8  Sep  6 2022  1:19PM  245754        10  SOTAH0000772   
9  Sep  6 2022  1:19PM  245754        10  SONSB0001925   

                        Customer ShipmentStatus  
0                      Hush Hush       Released  
1                      Hush Hush       Released  
2                      Hush Hush       Released  
3                      Hush Hush       Released  
4                      Hush Hush       Released  
5                      Hush Hush       Released  
6               Methapharm, Inc.      Cancelled  
7  Nextsource Biotechnology, LLC       Released  
8  Nextsource Biotechnology, LLC       Released  
9  Nextsource Biotechnology, LLC       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
28  245751       Released         67
29  245753       Released          1
30  245754       Released          3
31  245758      Cancelled          1
32  245759       Released          6

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Cancelled  Executing  Released
id                                            
245751                NaN        NaN      67.0
245753                NaN        NaN       1.0
245754                NaN        NaN       3.0
245758                1.0        NaN       NaN
245759                NaN        NaN       6.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Cancelled  Executing  Released
id                                            
245647                0.0        0.0       1.0
245654                0.0        1.0       0.0
245659                0.0        0.0       1.0
245689                0.0        0.0       7.0
245704                0.0        1.0       0.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Cancelled  Executing  Released
id                                            
245647                0.0          0         1
245654                0.0          1         0
245659                0.0          0         1
245689                0.0          0         7
245704                0.0          1         0

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Cancelled  Executing  Released
0               245647        0.0          0         1
1               245654        0.0          1         0
2               245659        0.0          0         1
3               245689        0.0          0         7
4               245704        0.0          1         0

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id  Cancelled Executing Released
0               245647        0.0                  1
1               245654        0.0         1         
2               245659        0.0                  1
3               245689        0.0                  7
4               245704        0.0         1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                       Customer
0    Sep  6 2022  1:31PM  245759        12                      Hush Hush
6    Sep  6 2022  1:27PM  245758        10               Methapharm, Inc.
7    Sep  6 2022  1:20PM  245753        10  Nextsource Biotechnology, LLC
8    Sep  6 2022  1:19PM  245754        10  Nextsource Biotechnology, LLC
11   Sep  6 2022  1:14PM  245724        15           Person & Covey, Inc.
32   Sep  6 2022  1:05PM  245751        10              ISDIN Corporation
99   Sep  6 2022  1:04PM  245749        10              ISDIN Corporation
150  Sep  6 2022  1:00PM  245746        10              ISDIN Corporation
197  Sep  6 2022 12:55PM  245750        21              NBTY Global, Inc.
198  Sep  6 2022 12:51PM  245743        10              ISDIN Corporation

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                       Customer  \
0  Sep  6 2022  1:31PM  245759        12                      Hush Hush   
1  Sep  6 2022  1:27PM  245758        10               Methapharm, Inc.   
2  Sep  6 2022  1:20PM  245753        10  Nextsource Biotechnology, LLC   
3  Sep  6 2022  1:19PM  245754        10  Nextsource Biotechnology, LLC   
4  Sep  6 2022  1:14PM  245724        15           Person & Covey, Inc.   
5  Sep  6 2022  1:05PM  245751        10              ISDIN Corporation   
6  Sep  6 2022  1:04PM  245749        10              ISDIN Corporation   
7  Sep  6 2022  1:00PM  245746        10              ISDIN Corporation   
8  Sep  6 2022 12:55PM  245750        21              NBTY Global, Inc.   
9  Sep  6 2022 12:51PM  245743        10              ISDIN Corporation   

   Cancelled Executing Released  
0        0.0                  6  
1        1.0                     
2        0.0                  1  
3        0.0                  3  
4        0.0                 21  
5        0.0                 67  
6        0.0                 51  
7        0.0                 47  
8        0.0         1           
9        0.0                 61

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                       Customer  \
0  Sep  6 2022  1:31PM  245759        12                      Hush Hush   
1  Sep  6 2022  1:27PM  245758        10               Methapharm, Inc.   
2  Sep  6 2022  1:20PM  245753        10  Nextsource Biotechnology, LLC   
3  Sep  6 2022  1:19PM  245754        10  Nextsource Biotechnology, LLC   
4  Sep  6 2022  1:14PM  245724        15           Person & Covey, Inc.   
5  Sep  6 2022  1:05PM  245751        10              ISDIN Corporation   
6  Sep  6 2022  1:04PM  245749        10              ISDIN Corporation   
7  Sep  6 2022  1:00PM  245746        10              ISDIN Corporation   
8  Sep  6 2022 12:55PM  245750        21              NBTY Global, Inc.   
9  Sep  6 2022 12:51PM  245743        10              ISDIN Corporation   

  Released Executing  
0        6            
1                     
2        1            
3        3            
4       21            
5       67            
6       51            
7       47            
8                  1  
9       61

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                       Customer  \
0  Sep  6 2022  1:31PM  245759        12                      Hush Hush   
1  Sep  6 2022  1:27PM  245758        10               Methapharm, Inc.   
2  Sep  6 2022  1:20PM  245753        10  Nextsource Biotechnology, LLC   
3  Sep  6 2022  1:19PM  245754        10  Nextsource Biotechnology, LLC   
4  Sep  6 2022  1:14PM  245724        15           Person & Covey, Inc.   

  Released Executing  
0        6            
1                     
2        1            
3        3            
4       21

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Date      id Warehouse                       Customer  \
0  Sep  6 2022  1:31PM  245759        12                      Hush Hush   
1  Sep  6 2022  1:27PM  245758        10               Methapharm, Inc.   
2  Sep  6 2022  1:20PM  245753        10  Nextsource Biotechnology, LLC   
3  Sep  6 2022  1:19PM  245754        10  Nextsource Biotechnology, LLC   
4  Sep  6 2022  1:14PM  245724        15           Person & Covey, Inc.   

   Released  Executing  
0       6.0        NaN  
1       NaN        NaN  
2       1.0        NaN  
3       3.0        NaN  
4      21.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                       Customer  \
0  Sep  6 2022  1:31PM  245759        12                      Hush Hush   
1  Sep  6 2022  1:27PM  245758        10               Methapharm, Inc.   
2  Sep  6 2022  1:20PM  245753        10  Nextsource Biotechnology, LLC   
3  Sep  6 2022  1:19PM  245754        10  Nextsource Biotechnology, LLC   
4  Sep  6 2022  1:14PM  245724        15           Person & Covey, Inc.   

   Released  Executing  
0       6.0        0.0  
1       0.0        0.0  
2       1.0        0.0  
3       3.0        0.0  
4      21.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                              
10         3194620     273.0       47.0
12          491463       6.0        1.0
15          245724      21.0        0.0
16          491463       4.0        0.0
19          491480      18.0       11.0
20          737102      12.0       13.0
21         1474154       4.0        2.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing
0        10  3194620     273.0       47.0
1        12   491463       6.0        1.0
2        15   245724      21.0        0.0
3        16   491463       4.0        0.0
4        19   491480      18.0       11.0
5        20   737102      12.0       13.0
6        21  1474154       4.0        2.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10     273.0       47.0
1        12       6.0        1.0
2        15      21.0        0.0
3        16       4.0        0.0
4        19      18.0       11.0
5        20      12.0       13.0
6        21       4.0        2.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  273.0
1        12  Released    6.0
2        15  Released   21.0
3        16  Released    4.0
4        19  Released   18.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10   12    15   16    19    20   21
Status                                           
Executing   47.0  1.0   0.0  0.0  11.0  13.0  2.0
Released   273.0  6.0  21.0  4.0  18.0  12.0  4.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10   12    15   16    19    20   21
0          Executing   47.0  1.0   0.0  0.0  11.0  13.0  2.0
1           Released  273.0  6.0  21.0  4.0  18.0  12.0  4.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10   12    15   16    19    20   21
0  Executing   47.0  1.0   0.0  0.0  11.0  13.0  2.0
1   Released  273.0  6.0  21.0  4.0  18.0  12.0  4.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()